In [2]:
!pip install pyspark spark-nlp

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 317.3/317.3 MB 3.5 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 620.8/620.8 kB 35.5 MB/s eta 0:00:00
  Created wheel for pyspark: filename=pyspark-3.5.3-py2.py3-none-any.whl size=317840625 sha256=678c271bab49f8519386bfd2be8d0b2afdaaca58e7123d5380dcd85b66ec0daf
  Stored in directory: /root/.cache/pip/wheels/1b/3a/92/28b93e2fbfdbb07509ca4d6f50c5e407f48dce4ddbda69a4ab
Successfully built pyspark


In [3]:
import sparknlp

from sparknlp.base import *
from sparknlp.annotator import *
from pyspark.ml import Pipeline

spark = sparknlp.start()

## .addException()

In [19]:
documenter = DocumentAssembler()\
.setInputCol("text")\
.setOutputCol("document")

tokenizer = Tokenizer()\
.setInputCols(["document"])\
.setOutputCol("token")\
.addException("Samsun")\
.addException("New York")\
.setCaseSensitiveExceptions(False)



pipeline = Pipeline(stages=[documenter, tokenizer])

text = "Samsun! is a city in Turkey."
text1 = "Peter Parker (Spiderman?) is a nice guy and lives in New York or new york but has no e-mail!"

df = spark.createDataFrame([[text]]).toDF("text")
df1 = spark.createDataFrame([[text1]]).toDF("text")

result = pipeline.fit(df).transform(df)
result1 = pipeline.fit(df1).transform(df1)

result.select("token.result").show(truncate=False)
result1.select("token.result").take(1)



+---------------------------------------+
|result                                 |
+---------------------------------------+
|[Samsun, !, is, a, city, in, Turkey, .]|
+---------------------------------------+



[Row(result=['Peter', 'Parker', '(', 'Spiderman', '?)', 'is', 'a', 'nice', 'guy', 'and', 'lives', 'in', 'New York', 'or', 'new york', 'but', 'has', 'no', 'e-mail', '!'])]

In [25]:
documenter = DocumentAssembler()\
.setInputCol("text")\
.setOutputCol("document")

tokenizer = Tokenizer() \
    .setInputCols(["document"]) \
    .setOutputCol("token") \
    .setSplitChars(['-']) \
    .setContextChars(['?', '!'])\
    .addException("New York")\
    .setCaseSensitiveExceptions(True)\
    .setSuffixPattern("([a])\z")\


pipeline = Pipeline(stages=[documenter, tokenizer])

text = "Samsun is a citya in Turkey. Another one is New York."
text1 = "Peter Parker (Spiderman?) is a nice guy and liv-es in New York or new york but has no e-mail!"

df = spark.createDataFrame([[text]]).toDF("text")
df1 = spark.createDataFrame([[text1]]).toDF("text")

result = pipeline.fit(df).transform(df)
result1 = pipeline.fit(df1).transform(df1)

result.select("token.result").show(truncate=False)
result1.select("token.result").take(1)



+------------------------------------------------------------------+
|result                                                            |
+------------------------------------------------------------------+
|[Samsun, is, a, city, a, in, Turkey., Another, one, is, New York.]|
+------------------------------------------------------------------+



[Row(result=['Peter', 'Parker', '(Spiderman?)', 'is', 'a', 'nice', 'guy', 'and', 'liv-es', 'in', 'New York', 'or', 'new', 'york', 'but', 'has', 'no', 'e-mail!'])]

In [28]:
documenter = DocumentAssembler()\
.setInputCol("text")\
.setOutputCol("document")

tokenizer = Tokenizer() \
    .setInputCols(["document"]) \
    .setOutputCol("token") \
    .setPrefixPattern("\A([a])")\
    .setSplitChars(['-']) \
    .setContextChars(['?', '!'])\
    .addException("New York")\
    .setCaseSensitiveExceptions(True)


pipeline = Pipeline(stages=[documenter, tokenizer])

text = "Samsun is a citya in Turkey. another one is New York.Another one is Paris."
text1 = "Peter Parker (Spiderman?) is a nice guy and liv-es in New York or new york but has no e-mail!"

df = spark.createDataFrame([[text]]).toDF("text")
df1 = spark.createDataFrame([[text1]]).toDF("text")

result = pipeline.fit(df).transform(df)
result1 = pipeline.fit(df1).transform(df1)

result.select("token.result").show(truncate=False)
result1.select("token.result").take(1)



+------------------------------------------------------------------------------------------+
|result                                                                                    |
+------------------------------------------------------------------------------------------+
|[Samsun, is, a, citya, in, Turkey., a, nother, one, is, New York.Another, one, is, Paris.]|
+------------------------------------------------------------------------------------------+



[Row(result=['Peter', 'Parker', '(Spiderman?)', 'is', 'a', 'nice', 'guy', 'a', 'nd', 'liv-es', 'in', 'New York', 'or', 'new', 'york', 'but', 'has', 'no', 'e-mail!'])]

In [30]:
documenter = DocumentAssembler()\
.setInputCol("text")\
.setOutputCol("document")

tokenizer = Tokenizer() \
    .setInputCols(["document"]) \
    .setOutputCol("token") \
    .setSplitChars(['-']) \
    .setContextChars(['?', '!'])\
    .addException("New York")\
    .setCaseSensitiveExceptions(True)\
    .setMinLength(2)\
    .setMaxLength(5)


pipeline = Pipeline(stages=[documenter, tokenizer])

text = "Samsun is a citya in Turkey. another one is New York.Another one is Paris."
text1 = "Peter Parker (Spiderman?) is a nice guy and liv-es in New York or new york but has no e-mail!"

df = spark.createDataFrame([[text]]).toDF("text")
df1 = spark.createDataFrame([[text1]]).toDF("text")

result = pipeline.fit(df).transform(df)
result1 = pipeline.fit(df1).transform(df1)

result.select("token.result").show(truncate=False)
result1.select("token.result").take(1)



+---------------------------------+
|result                           |
+---------------------------------+
|[is, citya, in, one, is, one, is]|
+---------------------------------+



[Row(result=['Peter', 'is', 'nice', 'guy', 'and', 'liv', 'es', 'in', 'or', 'new', 'york', 'but', 'has', 'no', 'mail'])]